In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/prem-odds-2021/2021 prem odds.csv
/kaggle/input/premresultsedited/results-EDITED.csv
/kaggle/input/2122premierleagueodds/prem odds condensed.csv
/kaggle/input/english-premier-league-results/results.parquet
/kaggle/input/english-premier-league-results/results.feather
/kaggle/input/english-premier-league-results/results.csv


fhjdsl

In [2]:
import pandas as pd
import numpy as np

path = 'Users/sebastianpriest/Dropbox/Seb/Data Projects/Premier League Results/'

#importing the data
raw_data = pd.read_csv('../input/premresultsedited/results-EDITED.csv', encoding = 'ISO-8859-1')

desirable_seasons = []
for i in range(4):
    yr = 22
    season = str(2000 + yr - 1 - i) + '-' + str(yr - i)
    desirable_seasons.append(season)
    
# Selecting data so only includes seasons 2011-12 to 2020-21 (inclusive)
data = raw_data.loc[raw_data["Season"].isin(desirable_seasons)]
data = data[["Season", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR"]]
data = data.reset_index(drop = True)


def score_pois_n(Home, Away, fixture_data, rev_fixture_data):
    n = 500
    def avg_goals_pois(Home, Away):
        
        
        avg_hg = 0.75 * (fixture_data["FTHG"].mean()) + 0.25 * (rev_fixture_data["FTAG"].mean())
        avg_ag = 0.75 * (fixture_data["FTAG"].mean()) + 0.25 * (rev_fixture_data["FTHG"].mean())
        
        pois_hg = np.random.poisson(avg_hg)
        pois_ag = np.random.poisson(avg_ag)

        return str(pois_hg) + "," + str(pois_ag)
    
    x = 0
    simulations = []
    while x < n:
        simulations.append(str(avg_goals_pois(Home,Away)))
        x += 1

    simulations = pd.DataFrame(simulations, columns = ["HG,AG"])
    freq_table = pd.crosstab(index=simulations["HG,AG"], columns = "Frequency")
    freq_table = freq_table.reset_index(level=0)
    
    freq_table["Probability"] = freq_table["Frequency"]/ sum(freq_table["Frequency"])
    freq_table["Percentage"] = freq_table["Probability"] * 100
    
    
    def hw_aw_or_d(score):
        hg_ag = score.split(",", 1)
        
        if hg_ag[0] > hg_ag[-1]:
            return "H"
        elif hg_ag[0] < hg_ag[-1]:
            return "A"
        else:
            return "D"
        
                
    freq_table["RESULT"] = freq_table["HG,AG"].apply(hw_aw_or_d)
    
    def predicted_odds(freq_table):
        
        hw_prob = freq_table.loc[freq_table["RESULT"] == "H"].Probability.sum()
        aw_prob = freq_table.loc[freq_table["RESULT"] == "A"].Probability.sum()
        d_prob = freq_table.loc[freq_table["RESULT"] == "D"].Probability.sum()
        
        results_table = pd.DataFrame([[(100 * hw_prob), (100 * aw_prob), (100 * d_prob)]],
                                     columns =["H", "A", "D"], 
                                     index = [ "Percentage"])
        results_table = results_table.round(2)
        

        return results_table
    
    odds_table = predicted_odds(freq_table)
    
    odds_table_trans = odds_table.transpose()
    
    most_prob = [odds_table_trans["Percentage"].idxmax(axis = 0), odds_table_trans["Percentage"].max(axis = 0)]
    
    
    
    return most_prob



def match_sim(Home, Away):
    fixture_data = data.loc[(data["HomeTeam"] == Home) & (data["AwayTeam"] == Away)]
    rev_fixture_data = data.loc[(data["HomeTeam"] == Away) & (data["AwayTeam"] == Home)]
    
    if len(fixture_data) < 3: 
        return [float("NaN"),float("NaN")]
    else:
        return score_pois_n(Home, Away, fixture_data, rev_fixture_data)
        


#match_sim(Home, Away, data)




#score_pois_n(Home, Away)

ARS = "Arsenal"
AVL = "Aston Villa"
BRE = "Brentford"
BRI = "Brighton"
BUR = "Burnley"

CHE = "Chelsea"
CRY = "Crystal Palace"
EVE = "Everton"
LEE = "Leeds"
LEI = "Leicester"

LIV = "Liverpool"
MCI = "Man City"
MUN = "Man United"
NEW = "Newcastle"
NOR = "Norwich"

SOU = "Southampton"
TOT = "Tottenham"
WAT = "Watford"
WHU = "West Ham"
WOL = "Wolves"

BOU = "Bournemouth"
FUL = "Fulham"


match_sim(MUN, CHE)

#for i in range(len(10)):
   # print(match_sim(Week_1[0], Week_1[1], data), 2 * "\n")
    
#for i in range(10):
    #print(match_sim(Week_2[i][0], Week_2[i][1], data), 2 * "\n")


['H', 59.0]

In [3]:


raw_odds_data_2122 = pd.read_csv('../input/2122premierleagueodds/prem odds condensed.csv')
raw_odds_data_2021 = pd.read_csv('../input/prem-odds-2021/2021 prem odds.csv')


def odds_file(raw_odds_data):

    odds_data =  raw_odds_data[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", 
                            "MaxH", "MaxD", "MaxA", "AvgH", "AvgD", "AvgA"]]
    

    home_wins = odds_data[odds_data.FTR == "H"]
    home_wins = home_wins[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "MaxH", "AvgH"]]
    home_wins = home_wins.rename(columns={'MaxH': 'MaxOdds', 'AvgH': 'AvgOdds'})

    away_wins = odds_data[odds_data.FTR == "A"]
    away_wins = away_wins[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "MaxA", "AvgA"]]
    away_wins = away_wins.rename(columns={'MaxA': 'MaxOdds', 'AvgA': 'AvgOdds'})

    draws = odds_data[odds_data.FTR == "D"]
    draws = draws[["Date", "HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "MaxD",  "AvgD"]]
    draws = draws.rename(columns={'MaxD': 'MaxOdds', 'AvgD': 'AvgOdds'})


    odds_data_2 = pd.concat([home_wins, away_wins, draws])
    

    odds_data_2["Out"] = odds_data_2.apply(lambda row : match_sim(row["HomeTeam"],row["AwayTeam"]), axis = 1)
    odds_data_2["PR"] = odds_data_2["Out"].apply(lambda x: x[0])
    odds_data_2["Perc"] = odds_data_2["Out"].apply(lambda x: x[1])
    del odds_data_2["Out"]
    return odds_data_2
#print(odds_data_2) 


odds_data_2 = odds_file(raw_odds_data_2122)

In [4]:

    
def tester_avg(row):
    stake = 10
    
    if (row["Perc"] >= 55) & (row["AvgOdds"] <= 3):
        if (row["PR"] == row["FTR"]) :
            winnings = (stake * row["AvgOdds"]) - stake
            return winnings
        else:
            winnings = - stake
            return winnings

def tester_max(row):
    stake = 10
    
    if (row["Perc"] >= 55) & (row["AvgOdds"] <= 3):
        if (row["PR"] == row["FTR"]) :
            winnings = (stake * row["MaxOdds"]) - stake
            return winnings
        else:
            winnings = - stake
            return winnings
def tester_max_2(row):
    stake = row["Perc"]
    
    if (row["Perc"] >= 0) & (row["MaxOdds"] <= 1000):
        if (row["PR"] == row["FTR"]) :
            winnings = (stake * row["MaxOdds"]) - stake
            return winnings
        else:
            winnings = - stake
            return winnings

def tester_comp(row):
    stake = 10
    
    if (row["Perc"] > 65 > (101 / row["MaxOdds"]) > 35):
        if (row["PR"] == row["FTR"]) :
            winnings = (stake * row["MaxOdds"]) - stake
            return winnings
        else:
            winnings = - stake
            return winnings
        


odds_data_2["Net"] =  odds_data_2.apply(lambda row : tester_avg(row), axis = 1)


total_net = round(odds_data_2["Net"].sum(), 2)
total_bets = odds_data_2["Net"].count()

print("predicted net profit is  £", total_net)

print("betting £10 each on  ", total_bets, " games" )

print("averaging a  ", round((10 * total_net / total_bets), 2), "%    return")
print(odds_data_2)

predicted net profit is  £ 554.2
betting £10 each on   96  games
averaging a   57.73 %    return
           Date     HomeTeam        AwayTeam  FTHG  FTAG FTR  MaxOdds  \
0    13/08/2021    Brentford         Arsenal     2     0   H     4.62   
1    14/08/2021   Man United           Leeds     5     1   H     1.59   
3    14/08/2021      Chelsea  Crystal Palace     3     0   H     1.30   
4    14/08/2021      Everton     Southampton     3     1   H     2.04   
5    14/08/2021    Leicester          Wolves     1     0   H     1.75   
..          ...          ...             ...   ...   ...  ..      ...   
360  15/05/2022        Leeds        Brighton     1     1   D     3.70   
362  15/05/2022     West Ham        Man City     2     2   D     5.55   
363  15/05/2022       Wolves         Norwich     1     1   D     4.80   
368  19/05/2022  Aston Villa         Burnley     1     1   D     3.70   
369  19/05/2022      Chelsea       Leicester     1     1   D     5.40   

     AvgOdds   PR  Perc  N